In [3]:
import os
os.listdir('../output/')

['freight_D_df.csv',
 'freight_I_df.csv',
 'movement_D_df.csv',
 'movement_I_df.csv',
 'passengers_D_df.csv',
 'passengers_I_df.csv']

In [5]:
import pandas as pd
import os
import numpy as np

path = "../output/"
filenamepaths = os.listdir(path)


def read_data(filepath):
    
    df = pd.read_csv(filepath, thousands=',',  dtype=np.float64)
#     df = pd.read_csv(filepath)

    # print(df.columns)
    df.rename(columns={'Unnamed: 0':'yearmon'}, inplace=True)
    # print(df.columns)
    return df


In [6]:

ALL_DF= {}
for f in filenamepaths:
    f = path + f
    df = read_data(f)
    ALL_DF[f[:-4]] = df
#     print(df.tail())
for x in ALL_DF.values():
    print(type(x))
print(type(ALL_DF['../output/passengers_D_df']), "\n")
print(ALL_DF['../output/passengers_D_df'].info())
ALL_DF['../output/freight_I_df'].tail()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   yearmon     52 non-null     float64
 1   EASTERN     52 non-null     float64
 2   WESTERN     52 non-null     float64
 3   SOUTHERN    52 non-null     float64
 4   NORTHERN    52 non-null     float64
 5   NORTH EAST  52 non-null     float64
dtypes: float64(6)
memory usage: 2.6 KB
None


,yearmon,EASTERN,WESTERN,SOUTHERN,NORTHERN,NORTH EAST
47,201911.0,4842.0,56507.0,58287.0,52807.0,0.0
48,202000.0,4295.0,52936.0,55385.0,50915.0,0.0
49,202001.0,3942.0,52947.0,56829.0,47815.0,0.0
50,202002.0,3394.0,40314.0,49013.0,40585.0,0.0
51,202003.0,280.0,14828.0,17012.0,7822.0,18.0


In [91]:

# df = ALL_DF[0]
# print(df.shape)
# pct_5 = int(0.1*len(df))
# df = df['EASTERN']
# df = df.diff()
# df.dropna(inplace=True)
# train, test = df.iloc[:len(df)-pct_5], df.iloc[-pct_5:]

# print(train.shape)
# print(np.asarray(train))
# print(test.shape)

In [92]:
# df = ALL_DF['passenger_D_I']
# df = df['NORTHERN']

def give_train_test(df, ratio):

    # df = df.diff()
    df.dropna(inplace=True)
    print(df.shape, type(df))
    pct_5 = int(ratio*len(df))

    train, test = df.iloc[:len(df)-pct_5], df.iloc[-pct_5:]
    return train, test


    


# df_arr = df[:]
# print("df_arr-",type(df_arr))
# print("train-",type(train))


In [93]:
# print(train.tail())
# plt.plot(train.index, train)
# plt.show()

In [94]:
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt

def plot_nums(train, test, forecast):
    plt.plot(train.index, train, 'r')
    indexx = [47+x for x in range(len(forecast))]
#     total = train.index + indexx
    plt.plot(indexx, forecast, 'g' )
    plt.plot(test.index, test, 'y')
#     plt.xticks([x for x in range(len(train)+len(forecast))], [total.index])
    plt.show()

    
def give_predictions(all_train, all_test, num_of_predictions):
    all_train_too = {}    

    for name, train_dfs in all_train.items():


        train_too_df = {}
        for ind_col in train_dfs.columns:

            if ind_col != 'yearmon':
                model = ARIMA(train_dfs[ind_col], order=(7,0,1))
                try:

                    model_fit = model.fit(disp=0)
                    # model_fit.summary()
                    forecast = model_fit.forecast(steps=num_of_predictions)[0]
    #                 len(forecast)
    #                 print(f"forecast: {name[10:]},{ind_col} with type: {type(forecast)}, {len(forecast)}")
                    forecast_df = pd.Series(forecast)
    #                 print(f"forecast: {name[10:]},{ind_col} with type: {type(forecast_df)}, {len(forecast_df)}")

    #                 print(type(forecast_df))
                    train_too_df[ind_col] = pd.concat([train_dfs[ind_col], forecast_df], ignore_index=True)
                    print(f"{len(forecast_df)}, {train_dfs[ind_col].shape}, {train_too_df[ind_col].shape} --- {name[10:]}, {ind_col}")
    #                 save_predictions()
    #                 plot_nums(train_dfs[ind_col], all_test[name][ind_col], forecast)
                except:
                    print(f"Not available -{name[10:]},{ind_col}")

        print(f"{name[10:]} = {type(train_too_df)}")
        all_train_too[name[10:]] = train_too_df
        
    return all_train_too


    # print(forecast)

In [95]:
print(len(all_train_too))
print(len(all_train_too['movement_D_df']))

6
5


In [101]:

all_train = {}
all_test = {}
for name, dfs in ALL_DF.items():
    
    train, test = give_train_test(dfs, 0.1)
    
    all_train[name] = train
    all_test[name] = test

    

all_train_too = give_predictions(all_train, all_test, 8 )


def save_predictions(all_train_too):
    for x, out1 in all_train_too.items():
        for in_col,in_df  in out1.items():
            in_df.to_csv(f'../forecast/{x}_{in_col}.csv')

(52, 6) <class 'pandas.core.frame.DataFrame'>
(52, 6) <class 'pandas.core.frame.DataFrame'>
(52, 6) <class 'pandas.core.frame.DataFrame'>
(52, 6) <class 'pandas.core.frame.DataFrame'>
(52, 6) <class 'pandas.core.frame.DataFrame'>
(52, 6) <class 'pandas.core.frame.DataFrame'>
8, (47,), (55,) --- passengers_D_df, EASTERN
8, (47,), (55,) --- passengers_D_df, WESTERN
8, (47,), (55,) --- passengers_D_df, SOUTHERN
8, (47,), (55,) --- passengers_D_df, NORTHERN
8, (47,), (55,) --- passengers_D_df, NORTH EAST
passengers_D_df = <class 'dict'>


/home/siddharth/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/siddharth/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


8, (47,), (55,) --- passengers_I_df, EASTERN


/home/siddharth/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/siddharth/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


8, (47,), (55,) --- passengers_I_df, WESTERN
8, (47,), (55,) --- passengers_I_df, SOUTHERN
8, (47,), (55,) --- passengers_I_df, NORTHERN


...

8, (47,), (55,) --- movement_D_df, SOUTHERN
8, (47,), (55,) --- movement_D_df, NORTHERN
8, (47,), (55,) --- movement_D_df, NORTH EAST
movement_D_df = <class 'dict'>


/home/siddharth/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


8, (47,), (55,) --- freight_I_df, EASTERN
Not available -freight_I_df,WESTERN


/home/siddharth/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/siddharth/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


8, (47,), (55,) --- freight_I_df, SOUTHERN
8, (47,), (55,) --- freight_I_df, NORTHERN
8, (47,), (55,) --- freight_I_df, NORTH EAST
freight_I_df = <class 'dict'>


In [97]:
# model = ARIMA(df_arr, order=(7,0,1))
# model_fit = model.fit(disp=0)
# # model_fit.summary()
# forecast = model_fit.forecast(steps=20)[0]
# print(forecast)

In [98]:
# plt.plot(df_arr.index, df_arr, 'r')
# indexx = [52+x for x in range(20)]
# plt.plot(indexx, forecast, 'g' )
# plt.show()

In [102]:
save_predictions(all_train_too)